# Code Generator


In [3]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess


In [24]:
print("Hello, World!")

Hello, World!


In [25]:
load_dotenv(override=True)

groq_api_key = os.getenv('GROQ_API_KEY')


if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")


Groq API Key exists and begins gsk_


In [26]:
# Connect to client libraries

groq_url = "https://api.groq.com/openai/v1"
ollama_url = "http://localhost:11434/v1"


groq = OpenAI(api_key=groq_api_key, base_url=groq_url) 
ollama = OpenAI(api_key="ollama", base_url=ollama_url)


In [27]:
models = ["qwen2.5:3b", "llama3.2:1b", "gemma3:270m", "openai/gpt-oss-120b", "qwen/qwen3-32b", "meta-llama/llama-4-maverick-17b", "moonshotai/kimi-k2-instruct-0905", "openai/gpt-oss-20b"]

clients = { "openai/gpt-oss-120b": groq, "qwen/qwen3-32b": groq, "meta-llama/llama-4-maverick-17b-128e-instruct": groq, "moonshotai/kimi-k2-instruct-0905": groq, "openai/gpt-oss-20b": groq, "qwen2.5:3b": ollama, "llama3.2:1b": ollama, "gemma3:270m": ollama}

# Want to keep costs ultra-low? Replace this with models of your choice, using the examples from yesterday

In [28]:
from system_info import retrieve_system_info

system_info = retrieve_system_info()
system_info

{'os': {'system': 'Windows',
  'arch': 'AMD64',
  'release': '11',
  'version': '10.0.26200',
  'kernel': '11',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': 'x86_64-w64-mingw32'},
 'package_managers': ['winget'],
 'cpu': {'brand': '', 'cores_logical': 12, 'cores_physical': 10, 'simd': []},
 'toolchain': {'compilers': {'gcc': 'gcc.EXE (Rev2, Built by MSYS2 project) 14.2.0',
   'g++': 'g++.EXE (Rev2, Built by MSYS2 project) 14.2.0',
   'clang': '',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': ''},
  'linkers': {'ld_lld': ''}}}

In [29]:
# Use g++ (available on your system); system_info shows clang is not installed
compile_command = [
    "g++",
    "-std=c++17",
    "-Ofast",
    "-march=native",
    "-flto",
    "-fvisibility=hidden",
    "-DNDEBUG",
    "main.cpp",
    "-o",
    "main.exe"
]

run_command = ["./main.exe"] if os.name != "nt" else ["main.exe"]


In [30]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
{compile_command}
Respond only with C++ code.
Python code to port:

```python
{python}
```
"""

In [31]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [32]:
def write_output(cpp):
    with open("main.cpp", "w") as f:
        f.write(cpp)

In [33]:
def port(model, python):
    client = clients[model]
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)
    return reply

In [34]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [35]:
def run_python(code):
    globals_dict = {"__builtins__": __builtins__}

    buffer = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buffer

    try:
        exec(code, globals_dict)
        output = buffer.getvalue()
    except Exception as e:
        output = f"Error: {e}"
    finally:
        sys.stdout = old_stdout

    return output

In [41]:
# Update compile_and_run to return string output instead of printing
def compile_and_run_and_capture():
    output_log = ""
    try:
        # 1. Compile
        # Capture stderr in case of compilation errors
        comp_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
        output_log += "✅ Compilation Successful.\n"
        
        # 2. Run (3 times for consistency, as per your original logic)
        output_log += "🚀 Running benchmark (3 iterations)...\n"
        output_log += "-" * 30 + "\n"
        
        for i in range(3):
            # Run the executable
            run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
            output_log += f"Run #{i+1}:\n{run_result.stdout}\n"
            
        return output_log

    except subprocess.CalledProcessError as e:
        # Handle compilation or execution errors
        error_msg = f"❌ Error occurred:\n{e.stderr}"
        if hasattr(e, 'stdout') and e.stdout:
             error_msg += f"\nOutput before crash:\n{e.stdout}"
        return error_msg

# Wrapper to save manual edits from the UI before running
def save_compile_run_wrapper(cpp_code):
    write_output(cpp_code) # Uses your existing write_output function
    return compile_and_run_and_capture()

In [42]:
compile_and_run_and_capture()

'✅ Compilation Successful.\n🚀 Running benchmark (3 iterations)...\n------------------------------\nRun #1:\nResult: 3.141592656542\nExecution Time: 0.263195 seconds\n\nRun #2:\nResult: 3.141592656542\nExecution Time: 0.448771 seconds\n\nRun #3:\nResult: 3.141592656542\nExecution Time: 0.858004 seconds\n\n'

In [43]:
import gradio as gr

# Using a theme for a modern look
with gr.Blocks(theme=gr.themes.Soft(), title="Python to C++ Converter") as ui:
    
    gr.Markdown("""
    # ⚡ Python to C++ Transpiler & Benchmark
    Convert Python algorithms to high-performance C++ using LLMs and compare execution speeds directly.
    """)

    # --- Settings Row ---
    with gr.Row(variant="panel"):
        model_selector = gr.Dropdown(
            choices=models, 
            label="Select LLM Model", 
            value=models[0],
            info="Choose the model to perform the translation."
        )

    # --- Main Interface Columns ---
    with gr.Row():
        # LEFT COLUMN: PYTHON (Input & Baseline)
        with gr.Column():
            gr.Markdown("### 🐍 Python Source")
            python_input = gr.Code(
                language="python", 
                label="Input Code", 
                value=pi, 
                lines=20
            )
            
            with gr.Row():
                run_py_btn = gr.Button("▶ Run Python (Baseline)")
            
            python_output = gr.Textbox(
                label="Python Execution Output", 
                lines=8, 
                placeholder="Python output will appear here..."
            )

        # RIGHT COLUMN: C++ (Output & High Performance)
        with gr.Column():
            gr.Markdown("### ⚙️ C++ Generated")
            # Using gr.Code allows you to edit the C++ before running if needed
            cpp_output = gr.Code(
                language="cpp", 
                label="Generated C++ Code", 
                lines=20, 
                interactive=True 
            )
            
            with gr.Row():
                convert_btn = gr.Button("✨ Convert to C++", variant="primary")
                run_cpp_btn = gr.Button("⚡ Compile & Run C++", variant="secondary")
            
            cpp_result_output = gr.Textbox(
                label="C++ Compilation & Execution Output", 
                lines=8,
                placeholder="Benchmark results will appear here..."
            )

    # --- Event Wiring ---
    
    # 1. Run Python Code
    run_py_btn.click(
        fn=run_python, 
        inputs=[python_input], 
        outputs=[python_output]
    )
    
    # 2. Convert Python to C++ (calls your existing 'port' function)
    convert_btn.click(
        fn=port, 
        inputs=[model_selector, python_input], 
        outputs=[cpp_output]
    )
    
    # 3. Compile & Run C++ (Saves code from box -> Compiles -> Runs)
    run_cpp_btn.click(
        fn=save_compile_run_wrapper, 
        inputs=[cpp_output], 
        outputs=[cpp_result_output]
    )

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "d:\LEARNING\LLM_Engineering\.venv\Lib\site-packages\httpx\_transports\default.py", line 101, in map_httpcore_exceptions
    yield
  File "d:\LEARNING\LLM_Engineering\.venv\Lib\site-packages\httpx\_transports\default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\LEARNING\LLM_Engineering\.venv\Lib\site-packages\httpcore\_sync\connection_pool.py", line 256, in handle_request
    raise exc from None
  File "d:\LEARNING\LLM_Engineering\.venv\Lib\site-packages\httpcore\_sync\connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\LEARNING\LLM_Engineering\.venv\Lib\site-packages\httpcore\_sync\connection.py", line 101, in handle_request
    raise exc
  File "d:\LEARNING\LLM_Engineering\.venv\Lib\site-packages\httpcore\_sync\connection.py", line 78, in handle_request
    stream = self